In [ ]:
%pip install einops peft decord pytorchvideo

# Encoder

In [1]:
# %pip install einops peft decord pytorchvideo

In [1]:
from encoder import MultiModalEncoder
import torch
import torch.nn as nn

# Usage of the MultiModalEncoder
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = MultiModalEncoder(device=device)

# Example input dictionary
input_dict = {
    'audio': './samples/0.wav',
    'image': './LanguageBind/assets/image/0.jpg',
    'video': './samples/0.mp4'
}

# Forward pass
encoding = encoder(input_dict)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/home/runs/LangBind/languagebind/audio/processing

In [4]:
media_embeds = encoding['concatenated_encodings']

In [5]:
media_embeds.shape

torch.Size([1, 1187, 2560])

In [4]:
# %pip install --upgrade transformers 

In [1]:
from transformers import AutoTokenizer
from modeling_phi import PhiForCausalLM
from languagebind import to_device
import torch
from encoder import MultiModalEncoder
import torch
import torch.nn as nn

torch.set_default_device("cuda")

class Pipeline(nn.Module):
    def __init__(self,torch_dtype=torch.float16,device='cuda'):
        super().__init__()
        self.encoder = MultiModalEncoder(device,torch_dtype=torch_dtype)
        self.encoder.freeze_encoders()
        self.llm = PhiForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True,torch_dtype=torch_dtype)
        self.device = device
    
    def forward(self, inputs):
        encoding = self.encoder(inputs) # text is ignored internally
        prompt = self.tokenizer(inputs['prompt'], return_tensors="pt")
        prompt = to_device(prompt, self.device)
        prompt_embeds = self.llm.transformer.embd(prompt['input_ids'])
        if encoding['concatenated_encodings'] is not None:
            input_embeds = torch.concat([encoding['concatenated_encodings'],prompt_embeds],dim=1)
        else:
            input_embeds = prompt_embeds
        outputs = self.llm(input_embeds=input_embeds,input_ids=None,bos_token_id=self.tokenizer.bos_token_id)
        return outputs

    def generate(self,inputs,max_length=10):
        encoding = self.encoder(inputs) # text is ignored internally
        prompt = self.tokenizer(inputs['prompt'], return_tensors="pt")
        prompt = to_device(prompt, self.device)
        prompt_embeds = self.llm.transformer.embd(prompt['input_ids'])
        if encoding['concatenated_encodings'] is not None:
            input_embeds = torch.concat([encoding['concatenated_encodings'],prompt_embeds],dim=1)
        else:
            input_embeds = prompt_embeds

        # Initialize past_key_values
        past_key_values = None
        generated_tokens = []

        returns = self.llm(input_embeds=input_embeds,inputs=None, past_key_values=past_key_values,input_ids=None)
        next_token_logits = returns.logits[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        generated_tokens.append(next_token)
        for step in range(max_length):
            returns = self.llm(inputs=next_token,input_ids=None)
            next_token_logits = returns.logits[:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            generated_tokens.append(next_token)
            # Check if the generation has finished
            if next_token.item() == self.tokenizer.eos_token_id:
                break
            break

        # Concatenate all generated tokens and decode into text
        generated_tokens = torch.cat(generated_tokens, dim=0)
        generated_text = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
        return generated_text

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/home/runs/LangBind/languagebind/audio/processing

In [ ]:
def generate(self,inputs,max_length=10):
    encoding = self.encoder(inputs) # text is ignored internally
    prompt = self.tokenizer(inputs['prompt'], return_tensors="pt")
    prompt = to_device(prompt, self.device)
    prompt_embeds = self.llm.transformer.embd(prompt['input_ids'])
    if encoding['concatenated_encodings'] is not None:
        input_embeds = torch.concat([encoding['concatenated_encodings'],prompt_embeds],dim=1)
    else:
        input_embeds = prompt_embeds

    # Initialize past_key_values
    past_key_values = None
    generated_tokens = []

    returns = self.llm(input_embeds=input_embeds,inputs=None, past_key_values=past_key_values,input_ids=None)
    next_token_logits = returns.logits[:, -1, :]
    next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
    generated_tokens.append(next_token)
    for step in range(max_length):
        returns = self.llm(inputs=next_token,input_ids=None)
        next_token_logits = returns.logits[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        generated_tokens.append(next_token)
        # Check if the generation has finished
        if next_token.item() == self.tokenizer.eos_token_id:
            break
        break

    # Concatenate all generated tokens and decode into text
    generated_tokens = torch.cat(generated_tokens, dim=0)
    generated_text = self.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

In [2]:
pipe = Pipeline()

In [1]:
input_dict = {
    'audio': './samples/0.wav',
    'video': './samples/0.mp4',
    'prompt': 'Apple, Orange, Pineapple, '
}
output = pipe.generate(input_dict)